In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.decomposition import PCA

In [2]:
# 降维
def pca_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension)
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,input_image.shape[0])
    pca_return_array = np.array(pca_return_array)
    #print (pca.explained_variance_ratio_)
    return pca_return_array

In [3]:
def trans(A):
    A_c = A.transpose(2,1,0)
    A_c = A_c.transpose(0,2,1)
    return A_c

In [4]:
# 原始高光谱数据
inp = loadmat('I:\data\Indian_pines_corrected.mat')['indian_pines_corrected']

In [5]:
# 转换坐标以后高光谱数据
inp_c = inp.transpose(2,1,0)
inp_c = inp_c.transpose(0,2,1)    

In [43]:
inp_c.shape

(200, 145, 145)

In [44]:
# 验证成功
s = list()
for i in range(inp.shape[0]):
    s.append(inp[i][:,0])
e = np.vstack(item for item in s)
e

array([[3172, 2580, 3687, ..., 2570, 3170, 3172],
       [2576, 2747, 2750, ..., 3859, 3686, 2744],
       [2744, 2576, 2744, ..., 2570, 2576, 2742],
       ...,
       [3324, 2983, 2988, ..., 2564, 2730, 3156],
       [3161, 2727, 2988, ..., 3156, 3159, 2561],
       [2979, 2977, 2814, ..., 3153, 3155, 3323]], dtype=uint16)

In [45]:
# 新的函数解法
def split_one_channel(A,channel):
    return A[channel-1]


In [46]:
# 根据pacth大小对数据进行镜像化操作
def mirror_matrix(A, fil):
    pad =int((fil-1)/2)
    for i in range(1,pad+1):
        up = np.array(A[2*i-1,:])
        down = np.array(A[-2*i,:])
        Matrix_up_down = np.vstack([up,A,down])
        left = np.array(Matrix_up_down[:,2*i-1]).reshape(len(Matrix_up_down),1)
        right = np.array(Matrix_up_down[:,-2*i]).reshape(len(Matrix_up_down),1)
        Matrix_left_right = np.hstack([left,Matrix_up_down,right,])
        A = Matrix_left_right
    return A
# 参数说明
# n   array size
# fil  filter size5*5
# pad = (fil-1)/2  padding大小

In [47]:
# s = split_one_channel(inp_c,1)
# s1 = mirror_matrix(s,3)

In [48]:
# 将镜像处理后的矩阵进行向量化操作
def split_array(A,fil):
    # 第一行
    row = len(A)-fil+1
    col = len(A[0])-fil+1
    s = []
    for j in range(col):
        temp = [0]*fil
        for z in range(fil):
            temp[z] = A[z,j:j+fil]
        com = np.concatenate(temp)
        s.append(com)
    s_array = np.array(s)
    for i in range(1,row): # row
        for j in range(col): # col
            temp = [0]* fil
            for z in range(fil):
                temp[z] = A[i+z,j:j+fil]
            com = np.concatenate(temp)
            s_array = np.vstack([s_array,com])
    return s_array

In [49]:
# s = split_one_channel(inp_c,1)
# s1 = mirror_matrix(s,3)
# s2 = split_array(s1,3)

In [50]:
# 计算欧几里得距离
def Euclidean(v1,v2):
    return np.linalg.norm(v1-v2)

In [51]:
# 计算单个patch的空间信息
def patch_spitial(v1, K_centroid):
    distance_array = np.array([Euclidean(v1, centroid) for centroid in K_centroid]) #距离matrix
    sums = np.sum(distance_array)
    average = np.average(distance_array)
    spitial_one_patch = np.maximum(0,average - distance_array) #依据公式求空间信息
    return spitial_one_patch

In [52]:
# 对一个channel的求解进行向量化
def channel_spitial(channel_matrix, K_centroid):
    channel_spitial_list = list()
    for i in range(len(channel_matrix)):
        channel_spitial_list.append(patch_spitial(channel_matrix[i], K_centroid))
    return np.array(channel_spitial_list)

#### 整体操作

In [59]:
inp_pca = pca_hpi(inp,10)
inp_pca_c = trans(inp_pca)
inp_pca_c.shape #后续测试的重点模块

(10, 145, 145)

In [41]:
## 构造n个通道的向量
def create_whole_vector(dim,fil,input_image):
    whole_origin_matrix_array = np.array((split_one_channel(input_image,i) for i in range(1,dim+1)))
    whole_mirror_matrix_array = np.array((mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim)))
    whole_split_matrix_array = np.array((split_array(whole_mirror_matrix_array[i],fil) for i in range(dim)))
    return whole_split_matrix_array
    

In [110]:
s = (split_one_channel(inp_pca_c,i) for i in range(1,11))
t = ((mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(10)))
p = np.array((split_array(whole_mirror_matrix_array[i],fil) for i in range(10)))

In [101]:
whole_origin_matrix_array = np.array(next(s))
while True:
    whole_origin_matrix_array = np.vstack((whole_origin_matrix_array,next(s)))

StopIteration: 

In [103]:
whole_origin_matrix_array = whole_origin_matrix_array.reshape(10,145,145)

In [104]:
whole_origin_matrix_array.shape

(10, 145, 145)

In [99]:
whole_origin_matrix_array = np.array([split_one_channel(inp_pca_c,i) for i in range(1,10+1)])
whole_origin_matrix_array.shape

(10, 145, 145)

In [111]:
whole_origin_matrix_array = np.array(next(s))
while True:
    try :
        whole_origin_matrix_array = np.vstack((whole_origin_matrix_array,next(s)))
    except StopIteration as e:
        whole_origin_matrix_array = whole_origin_matrix_array.reshape(10,145,145)
        break
whole_origin_matrix_array.shape

(10, 145, 145)

In [123]:
def create_whole_vector(dim,fil,input_image):
    # 生成器
    pad =int((fil-1)/2)
    s = (split_one_channel(input_image,i) for i in range(1,dim+1))
    whole_origin_matrix_array = np.array(next(s))
    while True:
        try:
            whole_origin_matrix_array = np.vstack((whole_origin_matrix_array,next(s)))
        except StopIteration as e:
            whole_origin_matrix_array = whole_origin_matrix_array.reshape(dim,input_image.shape[1],
                                                                          input_image.shape[2])                                                                     
            break
    p = (mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim))
    whole_mirror_matrix_array = np.array(next(p))
    while True:
        try:
            whole_mirror_matrix_array = np.vstack((whole_mirror_matrix_array, next(p)))
        except StopIteration as e:
            whole_mirror_matrix_array = whole_mirror_matrix_array.reshape(dim,input_image.shape[1]+pad*2,
                                                                          input_image.shape[2]+pad*2)
            break
    t = (split_array(whole_mirror_matrix_array[i],fil) for i in range(dim))
    whole_split_matrix_array = np.array(next(t))
    while True:
        try:
            whole_split_matrix_array = np.vstack((whole_split_matrix_array, next(t)))
        except StopIteration as e:
            whole_split_matrix_array = whole_split_matrix_array.reshape(dim,(input_image.shape[1])**2,-1)
            break
    return  whole_split_matrix_array

In [ ]:
test = create_whole_vector(10,5,inp_pca_c)
test.shape